In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pykat.commands import *
import pykat as pk
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'
from pylab import rcParams
rcParams['figure.figsize'] = 10, 8
rcParams.update({'font.size': 18})

                                              ..-
    PyKat 1.1.297         _                  '(
                          \`.|\.__...-""""-_." )
       ..+-----.._        /  ' `            .-'
   . '            `:      7/* _/._\    \   (
  (        '::;;+;;:      `-"' =" /,`"" `) /
  L.        \`:::a:f            c_/     n_'
  ..`--...___`.  .    ,
   `^-....____:   +.      www.gwoptics.org/pykat



In [2]:
# Grab data from COMSOL file and put it into arrays for analysis

data = pd.read_csv('EOBD_1_v.txt', delimiter='\s+', skiprows=9, header=None)
x = data[0]
y = data[1]
Ey = data[2]
Ex = data[3]

# the origin is defined at the bottom left corner of the EOBD, this is done to move the origion to the center of the ensemble
x = np.subtract(x, 0.0025)
y = np.subtract(y, 0.002)

In [3]:
# Since I am no interested in the field that present in the electrodes, silicon and copper, I remove these data points.
n_x = []
for i in range(int(len(x))):
    if (x[i] >= -.002 and x[i] <= 0.00201) == True:
        n_x.append(i)
        
x_c = []
y_c = []
Ey_c = []
Ex_c = []

for j in n_x:
    x_c.append(x[j])
    y_c.append(y[j])
    Ey_c.append(Ey[j])
    Ex_c.append(Ex[j])

x_c = np.array(x_c)
y_c = np.array(y_c)
Ey_c = np.array(Ey_c)
Ex_c = np.array(Ex_c)

In [4]:
# this is the same as above, but in the y-direction
n_y = []
for i in range(int(len(x_c))):
    if (y_c[i] >= -.0020 and y_c[i] <= 0.0020) == True:
        n_y.append(i)
        
x_cc = []
y_cc = []
Ey_cc = []
Ex_cc = []

for j in n_y:
    x_cc.append(x_c[j])
    y_cc.append(y_c[j])
    Ey_cc.append(Ey_c[j])
    Ex_cc.append(Ex_c[j])

x_cc = np.array(x_cc)
y_cc = np.array(y_cc)
Ey_cc = np.array(Ey_cc)
Ex_cc = np.array(Ex_cc)

In [50]:
from scipy.interpolate import griddata

n0 = [1.774 for i in x_cc] # index of refraction for RTP in the y direction (y-cut)
r33 = 38.5e-12 # [pm/V] electro optic coefficient
n_E = [j+0.5*(j**3)*r33*i for i,j in zip(Ey_cc, n0)] #use the above arrays to generate the electrical field dependant index of refraction

# ------------------------------------------------------
dOPL = [i*40e-3 - 1.774*40e-3 for i in n_E]   # 1
#dOPL = [i*20e-3 - 1.774*20e-3 for i in n_E]  # 2

phase = [k*2*np.pi/1064e-9 for k in dOPL]

# A point of interest can be found here. During the initial simulation, we assumed that the beam interacts with the EOBD through the extent of the crystal.
# However, we concluded, at least with regards to the phase, that only the second half of the crystal need be considered. Or, in other words, the length can
# be reduced by a factor of 1/2. This becomes relevant when we increase the complexity.

# As it is easier to work with a grid of the data we use the generated phase map array and interpolate it over a grid.
X = np.linspace(-0.002, 0.002, 101)
Y = X

XX, YY = np.meshgrid(X, Y)
phase_m = griddata((x_cc, y_cc), phase, (XX, YY), method='cubic', fill_value=0)

In [51]:
def HG_mode_content(w0, z, beam_data, X, Y, n, m):
    ''' Fuction that will calculate the complex coupling coefficients for a beam, in the HG basis, up to a user defined mode order'''
    import pykat.optics.gaussian_beams as gb
    q = gb.BeamParam(w0=w0, z=z)
    
    for i in range(0, n+1):
        for j in range(0, m+1):
            HG_mode = gb.HG_mode(q, n=i, m=j)
            HG_field = HG_mode.Unm(X,Y)
            k_nm = np.sum(np.multiply(np.conj(beam_data), HG_field))*np.diff(X)[0]*np.diff(Y)[0]
            print('%i%i: Mag: %.8E     Ang: %.2F'  % (i, j, np.abs(k_nm), np.angle(k_nm, deg=True)))

In [52]:
import pykat.optics.gaussian_beams as gb

# Use pykat to generate a q parameter, the corresponding field profile, using the grid generated, and, in turn, the phase map of the EOBD is applied to the field 
q = gb.BeamParam(w0=250e-6, z=0)
beam = gb.HG_mode(q,n=0, m=0) 
HG_field = beam.Unm(X,Y)
EOBDXHG = np.multiply(np.exp(-1j*phase_m), HG_field)


HG_mode_content(250e-6, 0, EOBDXHG, X, Y, 3, 3)

00: Mag: 9.99999264E-01     Ang: 0.00
01: Mag: 1.21271846E-03     Ang: 90.00
02: Mag: 1.02009805E-06     Ang: -179.63
03: Mag: 9.63358261E-06     Ang: -90.00
10: Mag: 8.68894710E-09     Ang: 90.28
11: Mag: 3.39859230E-08     Ang: 90.01
12: Mag: 3.19358024E-09     Ang: -90.33
13: Mag: 1.87892618E-08     Ang: -89.97
20: Mag: 2.28656019E-08     Ang: 154.78
21: Mag: 1.67163413E-05     Ang: 90.00
22: Mag: 2.85311838E-08     Ang: 168.68
23: Mag: 4.62799688E-07     Ang: -90.00
30: Mag: 1.89981947E-08     Ang: 89.95
31: Mag: 1.28209307E-08     Ang: -90.06
32: Mag: 5.44933622E-09     Ang: -89.78
33: Mag: 5.41027136E-11     Ang: 57.40


In [53]:
# Now we attempt to increase the complexity of the simulation

def mode_scattering_matrix(w0, z, n_E, dl, X, Y, n, m):
    '''For a given electric field dependent index of refraction, produces a phase map for a defined distance, dl, this phase map is then used
       to produces a scattering matrix up to a user defined order.'''
    
    scatter_matrix = np.zeros((n+1, m+1), dtype=np.complex_)
    dOPL_m = (n_E - 1.774)*dl
    phase_map = dOPL_m * 2 * np.pi / 1064e-9
    
    import pykat.optics.gaussian_beams as gb
    
    q = gb.BeamParam(w0=w0, z=z)
    HG00 = gb.HG_mode(q, n=0, m=0)
    HG00_f = HG00.Unm(X, Y)
    EOBDXHG = np.multiply(np.exp(-1j*phase_map), HG00_f)
    
    for i in range(0, n+1):
        for j in range(0, m+1):
            HG_mode = gb.HG_mode(q, n=i, m=j)
            HG_field = HG_mode.Unm(X,Y)
            k_nm = np.vdot(EOBDXHG, HG_field)*np.diff(X)[0]*np.diff(Y)[0]
            scatter_matrix[i][j] = k_nm
            
    return scatter_matrix

In [56]:
# interpolate the index of refraction over the same grid that was defined previously
nE_m = griddata((x_cc, y_cc), n_E, (XX, YY), method='cubic', fill_value=0)

# create a symmetric z-array, which is used to create an array of q parameters that are used to produces an array of scattering matrices for each point 
# in the simulation.

w0 = 250e-6
zR = np.pi * w0**2 / 1064e-9
z = np.linspace(-20e-3, 20e-3, 101)
q = [gb.BeamParam(w0=w0, z=i) for i in z]
dl = z[2]-z[1]

# ------------------------------------------------------------------
#A = [mode_scattering_matrix(w0, i, nE_m, dl, X, Y, 3, 3) for i in z] # 1
A = [mode_scattering_matrix(w0, i, nE_m, i, X, Y, 3, 3) for i in z] # 2 

In [57]:
# Matrix multiply all items in the array of scattering matrix
B = A[0]

for i in range(len(A)-1):
    
    if i > (len(A)-1): break
    else: B = np.matmul(B, A[i+1])
        

tot = 0
n_1 = 0
m_1 = 0

# Print results from final scattering matrix
for i in B: 
    for j in i:
        tot = tot + np.abs(j)**2
        print('%s%s: Mag:%.5E     Phase:%.5F'% (str(n_1),str(m_1), np.abs(j), np.angle(j, deg=True)))
        m_1 +=1
    n_1 +=1
    m_1 = 0
        
print('Total:', tot)

# There are a few ways of running this notebook, starting where we first acquire the phase map
# 1,1: we use the full length of the crystal in the intial simple simulation and maintain an equal distance between the slices.
#   > this leads to a difference in the magnitude of the 01 coefficients of about a factor of 100
#
# 1,2: we use the full length of the crystal in the intial simple simulation and use the cumulative sum of the distance between the slices
#  > this results in about a factor of 2 difference between the 01 modes 
#
# 2, 2: we use half the length of the crystal in the initial simulation and the cumulative sum of the distances between the slices
#  > this gives us results that are comparable.

00: Mag:9.99994E-01     Phase:-0.00000
01: Mag:6.09907E-04     Phase:96.18547
02: Mag:2.57974E-07     Phase:-166.91410
03: Mag:4.90197E-06     Phase:-71.44354
10: Mag:4.49892E-09     Phase:-96.31916
11: Mag:2.74394E-12     Phase:-0.13369
12: Mag:1.16061E-15     Phase:96.76674
13: Mag:2.20537E-14     Phase:-167.76270
20: Mag:7.08180E-09     Phase:-150.76356
21: Mag:4.31926E-12     Phase:-54.57810
22: Mag:1.82693E-15     Phase:42.32234
23: Mag:3.47150E-14     Phase:137.79289
30: Mag:9.25563E-09     Phase:-108.52850
31: Mag:5.64510E-12     Phase:-12.34303
32: Mag:2.38773E-15     Phase:84.55740
33: Mag:4.53711E-14     Phase:-179.97204
Total: 0.9999876520380133
